In [0]:
# Define o caminho apontando para dentro da subpasta SalesLT
caminho_bronze = "abfss://bronze@sabinoanalyticsdatalake.dfs.core.windows.net/"	
caminho_silver = "abfss://silver@sabinoanalyticsdatalake.dfs.core.windows.net/"

# Lista os arquivos dentro dessa pasta específica
display(dbutils.fs.ls(caminho_bronze + 'SalesLT'))


In [0]:
table_name = []

for i in dbutils.fs.ls(caminho_bronze + 'SalesLT'):
    print(i.name.split('/')[0])
    table_name.append(i.name.split('/')[0])

In [0]:
table_name

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType
  
for i in table_name:
    path = caminho_bronze + 'SalesLT/' + i + '/' + i + '.parquet'
    df = spark.read.format('parquet').load(path)
    column = df.columns

    for col in column:
        if "Date" in col or "date" in col:
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), 'UTC'), 'yyyy-MM-dd'))
            
    output_path = caminho_silver + 'SalesLT/' + i + '/' 
    df.write.mode('overwrite').format('delta').save(output_path)


In [0]:
display(df)